Product Categories
 • Cleanser
 • Sunscreen
 • Lip care
 • Face mask
 • Moisturizer
 • Eye cream
 • Cleansing oil
 • Toner
 • Hand cream

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import re

In [10]:
headers = {
    "User-Agent": "Billbot",
    "from": "xd2192@stern.nyu.edu"
}

In [2]:
def get_specifications(url,dic_ebay):
    headers = {
        "User-Agent": "Billbot",
        "from": "xd2192@stern.nyu.edu"
    }


    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    dl_blocks = soup.find_all("dl", class_="ux-labels-values")

    titles = []
    contents = []

    for block in dl_blocks:
        title_tag = block.find("dt")
        content_tag = block.find("dd")

        if title_tag and content_tag:
            titles.append(title_tag.text.strip())
            contents.append(content_tag.text.strip())

    # ✅ 构建字典时顺便处理特殊字段（如 volume）
    result_dict = {}
    for key, value in zip(titles, contents):
        key_lower = key.lower()

        # ✅ 针对 volume 字段特殊处理
        if key_lower == "volume":
            import re
            match = re.search(r"([\d.]+)\s*(fl oz|g)", value.lower())
            if match:
                number = float(match.group(1))
                unit = match.group(2)
                if unit == "g":
                    number = round(number / 28, 2)
                result_dict[key_lower] = number
            else:
                result_dict[key_lower] = value  # 无法解析则保留原始文本
        else:
            result_dict[key_lower] = value
    dic_ebay["brand"].append(result_dict.get("brand", "N/A"))
    if "volume" in result_dict:
        dic_ebay["product_volume"].append(result_dict["volume"])
    else:
        dic_ebay["product_volume"].append("N/A")


    #delivery 时间
    target = soup.find("span", string="Estimated between ")
    if target:
        bold_date = target.find_next("span", class_="ux-textspans ux-textspans--BOLD")
        date_str = bold_date.text.strip()

        # 加上年份（假设是今年）
        this_year = datetime.today().year
        full_date_str = f"{date_str} {this_year}"  # 👉 改动：只在找到日期后才定义

        # 解析 datetime 对象
        estimated_date = datetime.strptime(full_date_str, "%a, %b %d %Y")
        today = datetime.today()

        # 计算差值
        days_diff = (estimated_date - today).days

        if days_diff <= 3:
            dic_ebay["delivery_time"].append(1)
        else:
            dic_ebay["delivery_time"].append(0)
    else:
        dic_ebay["delivery_time"].append(0)


    #review volume
    import re
    target_span = soup.find("span", string=re.compile(r"This item\s*\(\d+\)"))

    if target_span:
        match = re.search(r"\((\d+)\)", target_span.text)
        if match:
            dic_ebay["review_volume"].append(int(match.group(1)))
        else:
            dic_ebay["review_volume"].append("N/A")
    else:
        dic_ebay["review_volume"].append("N/A")

In [3]:
def clean_volume_list(volume_list):
    cleaned = []
    for v in volume_list:
        if v == "N/A":
            cleaned.append("N/A")
        elif isinstance(v, float) or isinstance(v, int):
            cleaned.append(v)
        else:
            # 用正则提取第一个数字部分（如从 "3.38oz" 中提取 3.38）
            import re
            match = re.search(r"(\d+(\.\d+)?)", str(v))
            if match:
                cleaned.append(float(match.group(1)))
            else:
                cleaned.append("N/A")
    return cleaned

In [ ]:
def get_info(page_number,search_query,dic_ebay):
    
    url = f"https://www.ebay.com/sch/i.html?_nkw={search_query}&_sacat=0&_from=R40&_pgn={page_number}"
    resp = requests.get(url, headers=headers)

    soup = BeautifulSoup(resp.text,"html.parser")

    items = soup.find_all("div",{"class":"s-item__info clearfix"})

    for item in items[2:]:
        if item.find("div",{"class":"s-item__subtitle"}): # throw "picked for you" products away
            price = item.find("span",{"class":"s-item__price"}).text
            if "to" not in price:
                dic_ebay['total_price'].append(price)

                name = item.find("span",{"role":"heading", "aria-level":"3"})
                dic_ebay['product_description'].append(name.text)

                # If there is a star-rating than output the rating, otherwise output N/A
                if item.find("div",{"class":"x-star-rating"}):
                    stars = item.find_all("svg")
                    full_stars = 0
                    half_stars = 0
                    for star in stars:
                        key_value = star.get("data-marko-key", "")

                        if "1-1-2-1" in key_value:
                            full_stars += 1
                        elif "1-1-2-0" in key_value:
                            half_stars += 1
                    total_rating = full_stars + 0.5 * half_stars
                    dic_ebay["review_ratings"].append(total_rating)
                else:
                    dic_ebay["review_ratings"].append("N/A")
                

                # 查找是否存在 class 为 STRIKETHROUGH 的 <span>，即原价
                original_price = item.find("span", class_="STRIKETHROUGH")
                
                if original_price:
                    dic_ebay["discounted"].append(1)  # 有打折
                else:
                    dic_ebay["discounted"].append(0)  # 没有打折

                # 检查是否含有“Free delivery”字样
                delivery_tag = item.find("span", class_="s-item__shipping s-item__logisticsCost")
                if delivery_tag and "free delivery" in delivery_tag.text.lower():
                    dic_ebay["delivery_fee"].append(1)
                else:
                    dic_ebay["delivery_fee"].append(0)

                # ⭐ 提取链接
                link_tag = item.find_parent("li").find("a", class_="s-item__link")
                if link_tag:
                    product_url = link_tag["href"]
                    dic_ebay["product_link"].append(product_url)
                    get_specifications(product_url, dic_ebay)
                else:
                    dic_ebay["product_link"].append("N/A")


                dic_ebay["unit_price"].append(0)

    dic_ebay["product_volume"] = clean_volume_list(dic_ebay["product_volume"])
    

In [18]:
def create_df(page_range, search_query):
    dic_ebay = {
        "product_description":[],
        "brand":[],
        "total_price":[],
        "unit_price":[],
        "review_ratings": [],
        "delivery_time":[],
        "delivery_fee":[],
        "discounted":[],
        "product_link":[],
        "product_volume":[],
        "review_volume":[]
    }
    for page_number in range(1,page_range+1):
        get_info(page_number,search_query,dic_ebay)
    
    return pd.DataFrame(dic_ebay)

In [ ]:
def adjust_df(df):
    columns_to_check = df.columns.difference(['review_ratings'])

    # 清理含有 "N/A" 的行（原地赋值）
    df = df[~df[columns_to_check].isin(['N/A']).any(axis=1)]

    # 去除 $ 符号并转为 float 类型
    df["total_price"] = df["total_price"].replace('[\$,]', '', regex=True).astype(float)

    # 计算 unit_price 并保留两位小数
    df["unit_price"] = (df["total_price"] / df["product_volume"])


    df.reset_index(drop=True, inplace=True)

    df = df.drop(columns=["product_volume"])
    df["platform"] = "eBay"


    # 定义列重命名映射关系
    renamed_columns = {
        'product_description': 'product_name',
        'total_price': 'current_price',
        'review_ratings': 'star_rating',
        'review_volume': 'review_counts',
        'discounted': 'is_on_sale',
        'delivery_fee': 'is_free_shipping',
        'delivery_time': 'is_3day_delivery'
    }

    # 重命名列名
    df_renamed = df.rename(columns=renamed_columns)

    # 添加产品类别列
    df_renamed['product_category'] = 'cleanser'

    # 指定新的列顺序
    column_order = [
        'product_name', 'brand', 'current_price', 'unit_price', 'star_rating',
        'review_counts', 'is_on_sale', 'is_free_shipping', 'is_3day_delivery', 'product_category',
        'product_link', 'platform'
    ]

    # 重排序并生成最终的 DataFrame
    df_final = df_renamed[column_order]

    df_final["unit_price"] = df_final["unit_price"].round(2)
    df_final["unit_price"] = df_final["unit_price"].apply(lambda x: f"{x:.2f}")

    df_final['star_rating'] = df_final['star_rating'].replace("N/A", "3.0").astype(str)   

    return df_final 


<>:8: SyntaxWarning: invalid escape sequence '\$'
<>:8: SyntaxWarning: invalid escape sequence '\$'
/var/folders/c0/v7n5qhld2kd1zd7cmjk11_dc0000gn/T/ipykernel_22049/4006344134.py:8: SyntaxWarning: invalid escape sequence '\$'
  df["total_price"] = df["total_price"].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
cleanser_df = create_df(8, "cleanser")
df_cleanser_final = adjust_df(cleanser_df)

/var/folders/c0/v7n5qhld2kd1zd7cmjk11_dc0000gn/T/ipykernel_22049/254117418.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total_price"] = df["total_price"].replace('[\$,]', '', regex=True).astype(float)
/var/folders/c0/v7n5qhld2kd1zd7cmjk11_dc0000gn/T/ipykernel_22049/254117418.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unit_price"] = (df["total_price"] / df["product_volume"])


In [ ]:
sunscreen_df = create_df(8, "sunscreen")
df_sunscreen_final = adjust_df(sunscreen_df)

In [ ]:
lipcare_df = lipcare_df(8, "lip+care")
df_lipcare_final = adjust_df(lipcare_df)

In [ ]:
facemask_df = create_df(8, "face+mask+skin+care")
df_facemask_final = adjust_df(facemask_df)
df["unit_price"] = "N/A"

['N/A', 'N/A', '3.5ml x 8EA (28ml)', 'N/A', 'N/A', 'N/A', 3.5, 'N/A', 'N/A', 1.43, 2.53, 'N/A', 'N/A', 'N/A', 4.2, 1.0, 2.65, 'N/A', 2.65, '1 Box - 8 Packs', 0.89, '30pair(90 pcs)', 1.9, 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 1.6, 'N/A', 2.53, 'N/A', 'N/A', 1.7, 'N/A', 'see description', '30pcs', 'N/A', 0.89, 2.53, 'N/A', '75ml', 0.1, '50 ml', 6.0, 'N/A', 8.8, 3.0, 10.6, 'N/A', '4.23 oz', 'N/A', '20ml', 'N/A', '1.0 Ounce', 'N/A', '10sheets', 'N/A', 'Lot of 500', 'N/A', '1 Box - 8 Packs', 0.25, '20ml', 0.89, 'N/A', 'Not Available', '3', 'N/A']
['N/A', 'N/A', 3.5, 'N/A', 'N/A', 'N/A', 3.5, 'N/A', 'N/A', 1.43, 2.53, 'N/A', 'N/A', 'N/A', 4.2, 1.0, 2.65, 'N/A', 2.65, 1.0, 0.89, 30.0, 1.9, 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 1.6, 'N/A', 2.53, 'N/A', 'N/A', 1.7, 'N/A', 'N/A', 30.0, 'N/A', 0.89, 2.53, 'N/A', 75.0, 0.1, 50.0, 6.0, 'N/A', 8.8, 3.0, 10.6, 'N/A', 4.23, 'N/A', 20.0, 'N/A', 1.0, 'N/A', 10.0, 'N/A', 500.0, 'N/A', 1.0, 0.25, 20.0, 0.89, 'N/A', 'N/A', 3.0, 'N/A',

/var/folders/c0/v7n5qhld2kd1zd7cmjk11_dc0000gn/T/ipykernel_22049/4006344134.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total_price"] = df["total_price"].replace('[\$,]', '', regex=True).astype(float)
/var/folders/c0/v7n5qhld2kd1zd7cmjk11_dc0000gn/T/ipykernel_22049/4006344134.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unit_price"] = df.apply(


In [ ]:
moisturizer_df = create_df(8, "moisturizer")
df_moisturizer_final = adjust_df(moisturizer_df)

In [ ]:
eyecream_df = create_df(8, "eye+cream")
df_eyecream_final = adjust_df(eyecream_df)

In [ ]:
cleansingoil_df = create_df(8, "cleansing+oil")
df_cleansingoil_final = adjust_df(cleansingoil_df)

In [ ]:
toner_df = create_df(8, "toner")
df_toner_final = adjust_df(toner_df)

In [ ]:
handcream_df = create_df(8, "hand+cream")
df_handcream_final = adjust_df(handcream_df)

In [ ]:
#去除product_name里的表情符
def remove_non_ascii(text):
    return ''.join(i for i in str(text) if ord(i) < 128)

In [ ]:
df_lis=[]
df_lis.append(df_cleanser_final)
df_lis.append(df_cleansingoil_final)
df_lis.append(df_eyecream_final)
df_lis.append(df_facemask_final)
df_lis.append(df_handcream_final)
df_lis.append(df_lipcare_final)
df_lis.append(df_moisturizer_final)
df_lis.append(df_sunscreen_final)
df_lis.append(df_toner_final)

import numpy as np
for df in df_lis:
    df["product_name"] = df["product_name"].apply(remove_non_ascii)
    df["star_rating"] = df["star_rating"].replace(["N/A", "nan", np.nan], 3.0).astype(float)#将没有分数的rating统一成3分

df_cleanser_final.to_csv("df_cleanser_final.csv", index=False)
df_cleansingoil_final.to_csv("df_cleansingoil_final.csv", index=False)
df_eyecream_final.to_csv("df_eyecream_final.csv", index=False)
df_facemask_final.to_csv("df_facemask_final.csv", index=False)
df_handcream_final.to_csv("df_handcream_final.csv", index=False)
df_lipcare_final.to_csv("df_lipcare_final.csv", index=False)
df_moisturizer_final.to_csv("df_moisturizer_final.csv", index=False)
df_sunscreen_final.to_csv("df_sunscreen_final.csv", index=False)
df_toner_final.to_csv("df_toner_final.csv", index=False)

